In [69]:
import tensorflow as tf
import numpy as np
import random
#from google.colab import drive
from sklearn.model_selection import train_test_split

In [70]:
with open('iris.csv', 'r') as f:
  temp = np.genfromtxt(f, dtype='f4', delimiter=',')
for i in range(temp.shape[0]):
    if temp[i][-1] == 1.0:
        temp[i][-1] = 0.0
    else:
        temp[i][-1] = 1.0

In [71]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = tf.constant(indices[i:min(i + batch_size, num_examples)])
        yield tf.gather(features, j), tf.gather(labels, j)

In [72]:
# Optimization algorithm Stochastic Gradient Descent
def sgd(param, grad, lr, batch_size):
  param.assign_sub(lr * grad)

In [73]:
# Loss Function
def cross_entropy(z, t):
  return tf.keras.losses.CategoricalCrossentropy()(t,z)
  #return  -(1/len(z))*tf.reduce_sum(tf.math.log(z)*t)


In [74]:
def model(x,w):
  # layer with softmax function
  u = np.hstack((np.ones((x.shape[0],1)), x))@w
  u_exp = tf.math.exp(u)
  z = u_exp/tf.reduce_sum(u_exp,axis=1,keepdims=True)
  # if softmax activation is to be used,
  # z = tf.nn.softmax(u)
  return z

In [75]:
# Data Prep

X = temp[:,0:-1]
# One-hot output layer encoding (label 1 = class 1, label 0 = not class 1
from sklearn.preprocessing import LabelBinarizer
LB = LabelBinarizer()
labels = LB.fit_transform(temp[:,-1])
labels = np.hstack((labels, 1 - labels))
print(labels.shape)

# K: Nsamples, d: featureDimension, N: Nclasses
K,d = X.shape
N = labels.shape[1]

# partition into 80/20% training/testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, labels, train_size=0.8, shuffle=True, random_state=3, stratify=labels)

(150, 2)


In [76]:
# Hyperparameters

batch_size = 10
lr = 0.03 # learning rate
num_epochs = 50

In [77]:
# Learning

w = tf.Variable(tf.random.normal(shape=(d+1,N)), trainable=True)
for epoch in range(num_epochs):
  for x, y in data_iter(batch_size, X_train, y_train):
    # Feed-forward model
    with tf.GradientTape() as g:
      l = cross_entropy(model(x, w),y)
    # Compute gradient on l with respect to w
    dw = g.gradient(l, w)
    # Update parameters using their gradient
    sgd(w, dw, lr, batch_size)
  # After one epoch, Evaluate resubstitution loss
  train_loss = cross_entropy(model(tf.constant(X_train), w),y_train)
  print(f'epoch {epoch + 1}, loss {float(tf.reduce_mean(train_loss)):f}')

epoch 1, loss 0.058622
epoch 2, loss 0.054919
epoch 3, loss 0.051538
epoch 4, loss 0.048665
epoch 5, loss 0.046013
epoch 6, loss 0.043711
epoch 7, loss 0.041699
epoch 8, loss 0.039713
epoch 9, loss 0.038012
epoch 10, loss 0.036433
epoch 11, loss 0.034989
epoch 12, loss 0.033642
epoch 13, loss 0.032404
epoch 14, loss 0.031259
epoch 15, loss 0.030200
epoch 16, loss 0.029227
epoch 17, loss 0.028276
epoch 18, loss 0.027413
epoch 19, loss 0.026605
epoch 20, loss 0.025854
epoch 21, loss 0.025118
epoch 22, loss 0.024471
epoch 23, loss 0.023798
epoch 24, loss 0.023190
epoch 25, loss 0.022611
epoch 26, loss 0.022062
epoch 27, loss 0.021544
epoch 28, loss 0.021044
epoch 29, loss 0.020571
epoch 30, loss 0.020118
epoch 31, loss 0.019686
epoch 32, loss 0.019278
epoch 33, loss 0.018885
epoch 34, loss 0.018502
epoch 35, loss 0.018138
epoch 36, loss 0.017787
epoch 37, loss 0.017452
epoch 38, loss 0.017130
epoch 39, loss 0.016820
epoch 40, loss 0.016522
epoch 41, loss 0.016233
epoch 42, loss 0.015956
e

In [78]:
# Performance Evaluation
# Z_test is the linear model output (real #)
Z_test = model(X_test,w)
# idx is an array consisting of the index of largest output of each sample
idx = np.argmax(Z_test,axis=1)
# y_pred is one hot encoding of the predicted output
y_pred = np.zeros((idx.size, N))
y_pred[np.arange(idx.size),idx] = 1
# Cmat is the confusion matrix
Cmat = y_test.T@y_pred
Accuracy = np.round(100*np.sum(np.diagonal(Cmat))/np.sum(Cmat),2)
print('Confusion Matrix= \n', Cmat)
print('Accuracy = ', Accuracy, '%' )

Confusion Matrix= 
 [[20.  0.]
 [ 0. 10.]]
Accuracy =  100.0 %
